In [ ]:
import os
import shutil
from pathlib import Path

os.chdir('/content/drive/MyDrive/COMS4771')
DATA_PATH = Path.cwd() / 'data'
RAW = DATA_PATH / 'raw'
PROCESSED = DATA_PATH / 'processed'
SUBMISSION = DATA_PATH / 'submission'

from utils import *

In [ ]:
random_seeds = [64, 102, 202, 302, 512, 1024, 2024, 128, 802, 902]

i = 1
for seed in random_seeds:
    try:
        shutil.rmtree('models')
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))

    os.mkdir('models')

    xdev, ydev, xtest = load_data(preprocess_flag=True)
    xtrain, ytrain, xval, yval, xtest = process_and_split(xdev=xdev, ydev=ydev, xtest=xtest, random_seed=seed)

    input_size = xtrain.shape[1]
    output_size = 1

    EPOCHS = 100
    batch_size = 764
    learning_rate = 0.001

    train_loader, valid_loader = make_data_loader(xtrain=xtrain, ytrain=ytrain, xval=xval, yval=yval, batch_size=batch_size)
    
    model = build_model(input_size=input_size, output_size=1)

    print(f'\n\nProcessing seed: {i}/{10}\n\n')
    valid_loss = train(model=model, train_loader=train_loader, valid_loader=valid_loader, learning_rate=learning_rate, EPOCHS=EPOCHS, batch_size=batch_size, train_shape=xtrain.shape[0])

    file_name = f'ff_{i}.csv'
    ytest = predict(model=model, xtest=xtest, valid_loss=valid_loss, file_name=file_name)
    i += 1




Processing seed: 1/10


EPOCH:1/100 - Training Loss: 402.175869488773, Validation Loss: 345.9002483549572
EPOCH:2/100 - Training Loss: 351.79382356994194, Validation Loss: 328.77528424944194
EPOCH:3/100 - Training Loss: 346.9573288145953, Validation Loss: 321.7416996547154
EPOCH:4/100 - Training Loss: 341.8408741006419, Validation Loss: 322.12405642554876
EPOCH:5/100 - Training Loss: 339.5811480610923, Validation Loss: 320.3013435000465
EPOCH:6/100 - Training Loss: 337.52684538119735, Validation Loss: 319.139953031994
EPOCH:7/100 - Training Loss: 336.05298197525497, Validation Loss: 318.3755437941778
EPOCH:8/100 - Training Loss: 332.9037114691905, Validation Loss: 321.05441095261347
EPOCH:9/100 - Training Loss: 332.3155123544479, Validation Loss: 317.81788446335565
EPOCH:10/100 - Training Loss: 330.56265353473674, Validation Loss: 319.37373526436943
EPOCH:11/100 - Training Loss: 328.7152670630407, Validation Loss: 317.50487452915735
EPOCH:12/100 - Training Loss: 327.5830222701116, Va

In [ ]:
xtest = pd.read_csv(RAW / 'test_examples.csv')

files = [f'ff_{i}.csv' for i in range(1, 11)]
ensemble(files=files, test_size=xtest.shape[0], file_name='ensemble.csv')
